# Bonus Task

This notebook contains the solution for the task #3, management of a Virtual Power Plant with stochasticity. This solution differs from the solution from the textbook because it uses linear cost and utility functions. It is also simplified, with the power plant(s) always generating energy, instead of having the option of being shut down and then turned on again. <br> <br>

For reference:
<br>
Morales, J.M., Conejo, A.J., Madsen, H., Pinson, P. and Zugno, M., 2013. Integrating renewables in electricity
markets: operational problems (Vol. 205). Springer Science & Business Media., Chapter 8.

In [1]:
using JuMP, HiGHS, Plots, DataFrames

In [2]:
model = Model(HiGHS.Optimizer)
set_silent(model)

## Compose Model

### Variables and Functions

In [3]:
T = ["t1", "t2", "t3"] # Time steps
Ω = ["ω1", "ω2", "ω3"] # Scenarios

I = [ "i1", "i2"] # power plants

Q = ["q1"] # wind generation units

J = ["j1", "j2"] # flexible loads (energy consumers)

K = ["k1"] # storage units (batteries etc.)

λ = Dict("t1" => 20, "t2" => 80, "t3" => 45) # Market price 
π = Dict("ω1" => 0.2, "ω2" => 0.3, "ω3" => 0.5) # probabilities of scenarios

# Cost
EG_coeff = Dict("i1" => 30, "i2" => 20) # Linear coefficient of power plant cost function’
EG_intersect = Dict("i1" => 50, "i2" => 30) #’No-load cost’

# Utility
EL_coeff = Dict("j1" => 90, "j2" => 80) # Linear coeffcient of flexible load utility function 
EL_intersect = Dict("j1" => 5, "j2" => 20) # Constant term of flexible load utility function 

# Min and max power output of power plant i
PG_min = Dict( "i1" => 1, "i2" => 0.5)
PG_max = Dict("i1" => 5, "i2" => 3)

PG_0 = Dict("i1" => 2, "i2" => 1) # initial power output of power plant i
PG_ramp = Dict("i1" => 2, "i2" => 1) # max ramp of power output of power plant i

# Diesel set
C_diesel = 50 #Generating cost of the diesel set
Max_diesel = 0.5 #Capacity of the diesel set


# Min, max power consumption of flexible load j
PL_min = Dict( "j1" => 0.5, "j2" => 1)
PL_max = Dict("j1" => 2, "j2" => 3)
Min_Con = Dict("j1" => 2.5, "j2" => 4) # Minimum power consumption at the end of t3

PL_0 = Dict("j1" => 1.5, "j2" => 1.5 ) # initial power consumption of flexible load j
PL_ramp = Dict("j1" => 1, "j2" => 1) # max ramp of power consumption of flexible load j

# Min, max capacity of storage unit k
ES_max = Dict( "k1" => 1) 
ES_min = Dict( "k1" => 0.2) 
# Intital storage level
ES_0 = Dict( "k1" => 0.4) 
# (Dis)charging power limit
PS_charge_max = Dict( "k1" => 0.3)
PS_discharge_max = Dict( "k1" => 0.5)
PS_eff = Dict("k1" => 0.80) # Efficiency of (dis)charging storage unit k

# Wind generation values for the specific scenarios and time steps
PW_values = Dict(
    ("q1", "ω1", "t1") => 2.5, ("q1", "ω1", "t2") => 4.0, ("q1", "ω1", "t3") => 6.0,
    ("q1", "ω2", "t1") => 6.0, ("q1", "ω2", "t2") => 4.0, ("q1", "ω2", "t3") => 3.5,
    ("q1", "ω3", "t1") => 2.0, ("q1", "ω3", "t2") => 1.1, ("q1", "ω3", "t3") => 1.5
)

# Initialize an empty dictionary for PW
PW = Dict()

# Populate the PW dictionary with values from PW_values
for q in Q
    for ω in Ω
        for t in T
            PW[(q, ω, t)] = PW_values[(q, ω, t)]
        end
    end
end

In [4]:
# Auxiliary function to access values of previous time steps
function previous_time_period(t, T)
    index = findfirst(isequal(t), T)
    return T[index - 1]
end


previous_time_period (generic function with 1 method)

In [5]:
@variables(model, begin
        
    PD[T] # power exchanged with the electricity market at time step t
    EG[I,Ω,T] >= 0 # energy generated by power plant i in scenario ω and time step t
    EG_diesel[Ω,T] >= 0 # energy generated by the diesel set in scenario ω and time step t
    PG[I,Ω,T] >= 0 # power output by power plant i in scenario ω and time step t
    C[I,Ω,T] >= 0  # Cost function for each i, ω, t
    U[J,Ω,T] >= 0 # Utility function for each j, ω, t
    PW_curt[Q,Ω,T] >= 0 # curtailment of energy generated by wind generation unit q in scenario ω and time step t
    PS_discharge[K,Ω,T] >= 0 # power discharged from storage unit k in scenario ω and time step t
    PS_charge[K,Ω,T] >= 0 # power charged to storage unit k in scenario ω and time step t
    ES[K,Ω,T] >= 0 # energy status of storage unit k in scenario ω and time step t
    EL[J,Ω,T] >= 0 # energy consumed by flexible load j in scenario ω and time step t
    PL[J,Ω,T] >= 0 # load consumed by flexible load j in scenario ω and time step t
    
end)

(1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["t1", "t2", "t3"]
And data, a 3-element Vector{VariableRef}:
 PD[t1]
 PD[t2]
 PD[t3], 3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, ["i1", "i2"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 2×3×3 Array{VariableRef, 3}:
[:, :, "t1"] =
 EG[i1,ω1,t1]  EG[i1,ω2,t1]  EG[i1,ω3,t1]
 EG[i2,ω1,t1]  EG[i2,ω2,t1]  EG[i2,ω3,t1]

[:, :, "t2"] =
 EG[i1,ω1,t2]  EG[i1,ω2,t2]  EG[i1,ω3,t2]
 EG[i2,ω1,t2]  EG[i2,ω2,t2]  EG[i2,ω3,t2]

[:, :, "t3"] =
 EG[i1,ω1,t3]  EG[i1,ω2,t3]  EG[i1,ω3,t3]
 EG[i2,ω1,t3]  EG[i2,ω2,t3]  EG[i2,ω3,t3], 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["ω1", "ω2", "ω3"]
    Dimension 2, ["t1", "t2", "t3"]
And data, a 3×3 Matrix{VariableRef}:
 EG_diesel[ω1,t1]  EG_diesel[ω1,t2]  EG_diesel[ω1,t3]
 EG_diesel[ω2,t1]  EG_diesel[ω2,t2]  EG_diesel[ω2,t3]
 EG_diesel[ω3,t1]  EG_diesel[ω3,t2]  EG_

### Constraints

In [6]:
@constraints(model, begin
        
    # Energy balance constraint
    E1[ω in Ω, t in T], sum( EG[i,ω,t] for i in I) + EG_diesel[ω,t] + (sum( PW[q,ω,t] - PW_curt[q,ω,t] for q in Q) + sum( PS_discharge[k,ω,t] for k in K)) == sum( EL[j,ω,t] for j in J) + (sum( PS_charge[k,ω,t] for k in K) + PD[t])
    # Energy generation constraint of power plants 
    E2[i in I, ω in Ω, t in T], PG_min[i] <= PG[i,ω,t] 
    E3[i in I, ω in Ω, t in T], PG[i,ω,t] <= PG_max[i]  

    # Energy generation as average of power output of adjacent time steps
    E4[i in I, ω in Ω], EG[i,ω,"t1"] ==  (PG_0[i] + PG[i,ω,"t1"]) / 2
    E5[i in I, ω in Ω, t in T[2:end]], EG[i,ω,t] ==  (PG[i,ω,previous_time_period(t, T)] + PG[i,ω,t]) / 2

    # Max ramp up and down rates of power output

    E6[i in I, ω in Ω], PG_0[i] - PG[i,ω,"t1"] <= PG_ramp[i]
    E7[i in I, ω in Ω], PG[i,ω,"t1"] - PG_0[i] <= PG_ramp[i]
    E8[i in I, ω in Ω, t in T[2:end]], PG[i,ω,previous_time_period(t, T)] - PG[i,ω,t] <= PG_ramp[i]
    E9[i in I, ω in Ω, t in T[2:end]], PG[i,ω,t] - PG[i,ω,previous_time_period(t, T)] <= PG_ramp[i]

    # Diesel set max energy

    E10[ω in Ω, t in T], EG_diesel[ω,t] <= Max_diesel
   
        
    # Energy demand constraints of flexible loads  
    E11[j in J, ω in Ω, t in T], PL_min[j] <= PL[j,ω,t]  
    E12[j in J, ω in Ω, t in T], PL[j,ω,t] <= PL_max[j]
    E13[j in J, ω in Ω], sum(EL[j,ω,t] for t in T) >= Min_Con[j]


    # Energy generation as average of flexbile loads in adjacent time steps
    E14[j in J, ω in Ω], EL[j,ω,"t1"] ==  (PL_0[j] + PL[j,ω,"t1"]) / 2
    E15[j in J, ω in Ω, t in T[2:end]], EL[j,ω,t] ==  (PL[j,ω,previous_time_period(t, T)] + PL[j,ω,t]) / 2

    # Max ramp up and down rates of flexbile loads

    E16[j in J, ω in Ω], PL_0[j] - PL[j,ω,"t1"] <= PL_ramp[j]
    E17[j in J, ω in Ω], PL[j,ω,"t1"] - PL_0[j] <= PL_ramp[j]
    E18[j in J, ω in Ω, t in T[2:end]], PL[j,ω,previous_time_period(t, T)] - PL[j,ω,t] <= PL_ramp[j]
    E19[j in J, ω in Ω, t in T[2:end]], PL[j,ω,t] - PL[j,ω,previous_time_period(t, T)] <= PL_ramp[j]

    # Energy level of the battery

    # Consider battery SOC when starting
    E20[k in K, ω in Ω], ES[k,ω,"t1"] == ES_0[k] + (PS_eff[k] * PS_charge[k,ω,"t1"]) - ((1/PS_eff[k]) * PS_discharge[k,ω,"t1"])
    E21[k in K, ω in Ω, t in T[2:end]], ES[k, ω, t] == ES[k, ω, previous_time_period(t, T)] + (PS_eff[k] * PS_charge[k, ω, t]) - ((1/PS_eff[k]) * PS_discharge[k, ω, t])  
    # Max and min SOC
    E22[k in K, ω in Ω, t in T], ES_min[k] <= ES[k, ω, t]
    E23[k in K, ω in Ω, t in T], ES[k, ω, t] <= ES_max[k] 

    # Max and min (dis)charging rates

    E24[k in K, ω in Ω, t in T], 0 <= PS_charge[k, ω, t]
    E25[k in K, ω in Ω, t in T], PS_charge[k, ω, t] <= PS_charge_max[k]  
    E26[k in K, ω in Ω, t in T], 0 <= PS_discharge[k, ω, t]
    E27[k in K, ω in Ω, t in T], PS_discharge[k, ω, t] <= PS_discharge_max[k]

    # Maximum wind curtailment of wind power unit
    E28[q in Q, ω in Ω, t in T], PW_curt[q,ω,t] >= 0
    E29[q in Q, ω in Ω, t in T], PW_curt[q,ω,t] <= PW[q,ω,t] 
        
    # Power generated by wind power unit in different scenarios and time steps
    cost_function[i in I, ω in Ω, t in T], C[i, ω, t] == EG_coeff[i] * EG[i, ω, t] + EG_intersect[i]
    utility_function[j in J, ω in Ω, t in T], U[j, ω, t] == EL_coeff[j] * (EL[j, ω, t]) + EL_intersect[j]
        
    

end)

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, ["ω1", "ω2", "ω3"]
    Dimension 2, ["t1", "t2", "t3"]
And data, a 3×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 E1[ω1,t1] : -PD[t1] + EG[i1,ω1,t1] + EG[i2,ω1,t1] + EG_diesel[ω1,t1] - PW_curt[q1,ω1,t1] + PS_discharge[k1,ω1,t1] - PS_charge[k1,ω1,t1] - EL[j1,ω1,t1] - EL[j2,ω1,t1] == -2.5  …  E1[ω1,t3] : -PD[t3] + EG[i1,ω1,t3] + EG[i2,ω1,t3] + EG_diesel[ω1,t3] - PW_curt[q1,ω1,t3] + PS_discharge[k1,ω1,t3] - PS_charge[k1,ω1,t3] - EL[j1,ω1,t3] - EL[j2,ω1,t3] == -6
 E1[ω2,t1] : -PD[t1] + EG[i1,ω2,t1] + EG[i2,ω2,t1] + EG_diesel[ω2,t1] - PW_curt[q1,ω2,t1] + PS_discharge[k1,ω2,t1] - PS_charge[k1,ω2,t1] - EL[j1,ω2,t1] - EL[j2,ω2,t1] == -6       E1[ω2,t3] : -PD[t

### Optimization Objective

In [7]:
@objective(model, Max,
    sum(
    ( λ[t] * PD[t]) + sum(π[ω] * ( sum( U[j,ω,t] for j in J) - ( sum( C[i,ω,t] for i in I) + C_diesel * EG_diesel[ω,t])) for ω in Ω) for t in T))


20 PD[t1] + 0.2 U[j1,ω1,t1] + 0.2 U[j2,ω1,t1] - 0.2 C[i1,ω1,t1] - 0.2 C[i2,ω1,t1] - 10 EG_diesel[ω1,t1] + 0.3 U[j1,ω2,t1] + 0.3 U[j2,ω2,t1] - 0.3 C[i1,ω2,t1] - 0.3 C[i2,ω2,t1] - 15 EG_diesel[ω2,t1] + 0.5 U[j1,ω3,t1] + 0.5 U[j2,ω3,t1] - 0.5 C[i1,ω3,t1] - 0.5 C[i2,ω3,t1] - 25 EG_diesel[ω3,t1] + 80 PD[t2] + 0.2 U[j1,ω1,t2] + 0.2 U[j2,ω1,t2] - 0.2 C[i1,ω1,t2] - 0.2 C[i2,ω1,t2] - 10 EG_diesel[ω1,t2] + 0.3 U[j1,ω2,t2] + 0.3 U[j2,ω2,t2] - 0.3 C[i1,ω2,t2] - 0.3 C[i2,ω2,t2] - 15 EG_diesel[ω2,t2] + 0.5 U[j1,ω3,t2] + 0.5 U[j2,ω3,t2] - 0.5 C[i1,ω3,t2] - 0.5 C[i2,ω3,t2] - 25 EG_diesel[ω3,t2] + 45 PD[t3] + 0.2 U[j1,ω1,t3] + 0.2 U[j2,ω1,t3] - 0.2 C[i1,ω1,t3] - 0.2 C[i2,ω1,t3] - 10 EG_diesel[ω1,t3] + 0.3 U[j1,ω2,t3] + 0.3 U[j2,ω2,t3] - 0.3 C[i1,ω2,t3] - 0.3 C[i2,ω2,t3] - 15 EG_diesel[ω2,t3] + 0.5 U[j1,ω3,t3] + 0.5 U[j2,ω3,t3] - 0.5 C[i1,ω3,t3] - 0.5 C[i2,ω3,t3] - 25 EG_diesel[ω3,t3]

In [8]:
print(model)

## Optimization and Results

In [9]:
optimize!(model)

In [10]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [11]:
# Print all the important values of the optimized model
println(string("Objective Value: ", objective_value(model), "\n"))
for ω in Ω
    println(string("Scenario ", ω, ": \n"))
    for t in T
        println(string("Time Step ", t, ":"))
        println(string("Energy Exchanged with Market ", PD[t], " : ", value.(PD[t])))
        println("PV Power Generation and Curtailment : ")
        for q in Q
            println(string("\tPV Generation ", "PW[", q, ", ", ω, ", ", t, "] : ", value.(PW[q, ω, t])))
            println(string("\tPV Curtailment ", PW_curt[q, ω, t], " : ", value.(PW_curt[q, ω, t])))
        end
        println("Energy Generation: ")
        for i in I
            if(t == "t1")
                println(string("\tPG_0[", i, "] : ", value.(PG_0[i])))
            end
            println(string("\t", PG[i, ω, t], " : ", value.(PG[i, ω, t])))
            println(string("\t", EG[i, ω, t], " : ", value.(EG[i, ω, t])))
        end
        println(string("\t", EG_diesel[ω, t], " : ", value.(EG_diesel[ω, t])))
        println("Energy Consumption: ")
        for j in J
            if(t == "t1")
                println(string("\tPL_0[", j, "] : ", value.(PL_0[j])))
            end
            println(string("\t", PL[j, ω, t], " : ", value.(PL[j, ω, t])))
            println(string("\t", EL[j, ω, t], " : ", value.(EL[j, ω, t])))
        end
        println("Energy Storage: ")
        for k in K
            if(t == "t1")
                println(string("\tES_0[", k, "] : ", value.(ES_0[k]), " (Efficiency: ", PS_eff[k], ")"))
            end
            println(string("\t", PS_charge[k, ω, t], " : ", value.(PS_charge[k, ω, t])))
            println(string("\t", PS_discharge[k, ω, t], " : ", value.(PS_discharge[k, ω, t])))
            println(string("\t", ES[k, ω, t], " : ", value.(ES[k, ω, t])))

        end

        println("\n")
    end
    println()
end

Objective Value: 1137.6917073170732

Scenario ω1: 

Time Step t1:
Energy Exchanged with Market PD[t1] : 2.45
PV Power Generation and Curtailment : 
	PV Generation PW[q1, ω1, t1] : 2.5
	PV Curtailment PW_curt[q1,ω1,t1] : 0.0
Energy Generation: 
	PG_0[i1] : 2
	PG[i1,ω1,t1] : 2.8780487804878048
	EG[i1,ω1,t1] : 2.4390243902439024
	PG_0[i2] : 1
	PG[i2,ω1,t1] : 2.0
	EG[i2,ω1,t1] : 1.5
	EG_diesel[ω1,t1] : 0.0
Energy Consumption: 
	PL_0[j1] : 1.5
	PL[j1,ω1,t1] : 2.0
	EL[j1,ω1,t1] : 1.75
	PL_0[j2] : 1.5
	PL[j2,ω1,t1] : 2.5
	EL[j2,ω1,t1] : 2.0
Energy Storage: 
	ES_0[k1] : 0.4 (Efficiency: 0.8)
	PS_charge[k1,ω1,t1] : 0.2390243902439022
	PS_discharge[k1,ω1,t1] : 0.0
	ES[k1,ω1,t1] : 0.5912195121951218


Time Step t2:
Energy Exchanged with Market PD[t2] : 4.901999999999999
PV Power Generation and Curtailment : 
	PV Generation PW[q1, ω1, t2] : 4.0
	PV Curtailment PW_curt[q1,ω1,t2] : 0.0
Energy Generation: 
	PG[i1,ω1,t2] : 2.799999999999999
	EG[i1,ω1,t2] : 2.8390243902439014
	PG[i2,ω1,t2] : 3.0
	EG[i2